### <a id='top_of_page'>Dynamically Creating Calendar Mapping Table</a>

This script demonstrates:
1. How to dynamically create a calendar mapping table
2. How to append each calendar into a master dataframe using two different methods: pd.DataFrame.append() and pd.concat()
    - <i>** pd.DataFrame.append() was deprecated in pandas 1.4.0 onward</i>
    - Section links for these methods:
        - [Consolidate with pd.DataFrame.append()](#append)
        - [Consolidate with pd.concat()](#concat)

In [1]:
# Consolidate all imports here
import pandas as pd
import datetime as dt

In [2]:
# Version check pandas
print('pandas:', pd.__version__)

pandas: 1.3.4


In [3]:
# Create list of past 5 years
years = [year for year in range(dt.datetime.now().year - 5, dt.datetime.now().year + 1)]
years

[2017, 2018, 2019, 2020, 2021, 2022]

In [4]:
# Dynamically create a weekly calendar for each year as a dataframe, then store each individual calendar in a master dictionary

# Create master dictionary
individual_calendars = {}

# Create individual calendar inside the master dictionary
for i in years:
    individual_calendars[f'year_{i}'] = pd.DataFrame({
        'Week': pd.date_range(start=f'{i}-01-01', end=f'{i}-12-31').to_period('W-SAT').unique()
    })

In [5]:
# Check dictionary to ensure each calendar was created successfully
individual_calendars.keys()

dict_keys(['year_2017', 'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022'])

In [6]:
# Check one of the calendar dataframes
individual_calendars['year_2022'].head()

,Week
0,2021-12-26/2022-01-01
1,2022-01-02/2022-01-08
2,2022-01-09/2022-01-15
3,2022-01-16/2022-01-22
4,2022-01-23/2022-01-29


In [7]:
# Create a column in each calendar with a loop
for df in individual_calendars:
    individual_calendars[df]['Week Number'] = individual_calendars[df]['Week'].dt.week

In [8]:
# Check all dataframes to ensure column came through
for df in individual_calendars:
    print(individual_calendars[df].dtypes)
    print(len(individual_calendars[df].index))

Week           period[W-SAT]
Week Number            int64
dtype: object
53
Week           period[W-SAT]
Week Number            int64
dtype: object
53
Week           period[W-SAT]
Week Number            int64
dtype: object
53
Week           period[W-SAT]
Week Number            int64
dtype: object
53
Week           period[W-SAT]
Week Number            int64
dtype: object
53
Week           period[W-SAT]
Week Number            int64
dtype: object
53


In [9]:
# Validate collective total rows across all dataframes

rows_count_each = []

for df in individual_calendars:
    rows_count_each.append(len(individual_calendars[df].index))

print(rows_count_each)
print('Total row count:', sum(rows_count_each))

[53, 53, 53, 53, 53, 53]
Total row count: 318


#### <a id='append'>Consolidate Master DataFrame Using pd.DataFrame.append()</a>
<i>[** Go to top of page](#top_of_page)</i>

Can only use this method prior to pandas 1.4.0


In [10]:
# Instatiate master dataframe
df_master = pd.DataFrame({})

# Add all dictionaries to master dataframe
for key, data in individual_calendars.items():
    df_master = df_master.append(pd.concat([data]))

In [11]:
# Reset index and preview
df_master = df_master.reset_index(drop=True)
df_master

,Week,Week Number
0,2017-01-01/2017-01-07,1
1,2017-01-08/2017-01-14,2
2,2017-01-15/2017-01-21,3
3,2017-01-22/2017-01-28,4
4,2017-01-29/2017-02-04,5
...,...,...
313,2022-11-27/2022-12-03,48
314,2022-12-04/2022-12-10,49
315,2022-12-11/2022-12-17,50
316,2022-12-18/2022-12-24,51


#### <a id='concat'>Consolidate Master DataFrame Using pd.concat method</a>

<i>[** Go to top of page](#top_of_page)</i>

Must use this method for pandas 1.4.0 and later

In [12]:
# Instantiate lists to append dictionary data
week_list = []
week_number_list = []

In [13]:
# Append dictionary data into lists
for key, data in individual_calendars.items():
    week_list.append(data['Week'])
    week_number_list.append(data['Week Number'])

In [14]:
# Need to unnest the lists; store in separate list objects

week_list_unnested = []
for i in week_list:
    for j in i:
        week_list_unnested.append(j)

week_number_list_unnested = []
for i in week_number_list:
    for j in i:
        week_number_list_unnested.append(j)


In [15]:
# Validate list lengths
print(len(week_list_unnested))
print(len(week_number_list_unnested))

318
318


In [16]:
# Create master dataframe using unnested lists

df_master_2 = pd.DataFrame({
    'Week': week_list_unnested,
    'Week Number': week_number_list_unnested,
})

df_master_2

,Week,Week Number
0,2017-01-01/2017-01-07,1
1,2017-01-08/2017-01-14,2
2,2017-01-15/2017-01-21,3
3,2017-01-22/2017-01-28,4
4,2017-01-29/2017-02-04,5
...,...,...
313,2022-11-27/2022-12-03,48
314,2022-12-04/2022-12-10,49
315,2022-12-11/2022-12-17,50
316,2022-12-18/2022-12-24,51


In [17]:
# Remove duplicates in Week column - not known why there are duplicates here, but should investigate if using this in the future
df_master_2 = df_master_2.drop_duplicates(subset='Week', keep='first')
print('Remaining duplicates in Week column:', df_master_2['Week'].duplicated().any())
df_master_2

Remaining duplicates in Week column: False


,Week,Week Number
0,2017-01-01/2017-01-07,1
1,2017-01-08/2017-01-14,2
2,2017-01-15/2017-01-21,3
3,2017-01-22/2017-01-28,4
4,2017-01-29/2017-02-04,5
...,...,...
313,2022-11-27/2022-12-03,48
314,2022-12-04/2022-12-10,49
315,2022-12-11/2022-12-17,50
316,2022-12-18/2022-12-24,51


In [18]:
# Validate week number counts

df_master_2_value_counts = pd.DataFrame(df_master_2['Week Number'].value_counts())
df_master_2_value_counts = df_master_2_value_counts.reset_index()
df_master_2_value_counts = df_master_2_value_counts.rename(columns={'index': 'Week Number', 'Week Number': 'Count'})
df_master_2_value_counts = df_master_2_value_counts.sort_values(by='Week Number')
df_master_2_value_counts = df_master_2_value_counts.reset_index(drop=True)
df_master_2_value_counts

,Week Number,Count
0,1,6
1,2,6
2,3,6
3,4,6
4,5,6
5,6,6
6,7,6
7,8,6
8,9,6
9,10,6
